# AML_ Adaboost Model vs Class Imbalanced Data

**Experiment No : 8**: Consider the Attrition data. Fit a Decision tree Model to predict Attrition based on the other variables(use optimal hyperparameters). Use Random Forest model with optimal hyperparameters.Now use adaboost model with optimal hyperparameters to classify Attrition. Does the adaboost model provide any extra improvement over the random forest model and the decision tree model for this highly imbalance data.

### Importing Required Libraries

In [1]:
#import Required Libraries
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import time

#for train-test split 
from sklearn.model_selection import train_test_split

# For fitting regression tree
from sklearn.tree import DecisionTreeClassifier

#to compute cross validation score
from sklearn.model_selection import cross_val_score

#scores
from sklearn.metrics import confusion_matrix,f1_score, accuracy_score, classification_report, roc_auc_score
from sklearn.model_selection import GridSearchCV

import time

from sklearn.ensemble import RandomForestClassifier #import randomforest Classifier

from sklearn.ensemble import AdaBoostClassifier #import adaboostclassifer

In [2]:
import warnings
warnings.filterwarnings('ignore')

### Reading Data

In [4]:
att =pd.read_csv(r"C:\Users\Hi\Desktop\Praxis\IML\attrition.csv") # Reading csv files
att.head()                        # Display the top 5 rows of data

,Age,Attrition,BusinessTravel,Department,EducationField,EnvironmentSatisfaction,Gender,JobInvolvement,JobLevel,JobSatisfaction,MaritalStatus,MonthlyIncome,OverTime,TotalWorkingYears,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,DistanceFromHome
0,41,1,Travel_Rarely,Sales,Life Sciences,2,Female,3,2,4,Single,5993,Yes,8,1,6,4,0,5,1
1,49,0,Travel_Frequently,Research & Development,Life Sciences,3,Male,2,2,2,Married,5130,No,10,3,10,7,1,7,8
2,37,1,Travel_Rarely,Research & Development,Other,4,Male,2,1,3,Single,2090,Yes,7,3,0,0,0,0,2
3,33,0,Travel_Frequently,Research & Development,Life Sciences,4,Female,3,1,3,Married,2909,Yes,8,3,8,7,3,0,3
4,27,0,Travel_Rarely,Research & Development,Medical,1,Male,3,1,2,Married,3468,No,6,3,2,2,2,2,2


In [5]:
att.dtypes #No of rows and columns

Age                         int64
Attrition                   int64
BusinessTravel             object
Department                 object
EducationField             object
EnvironmentSatisfaction     int64
Gender                     object
JobInvolvement              int64
JobLevel                    int64
JobSatisfaction             int64
MaritalStatus              object
MonthlyIncome               int64
OverTime                   object
TotalWorkingYears           int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsInCurrentRole          int64
YearsSinceLastPromotion     int64
YearsWithCurrManager        int64
DistanceFromHome            int64
dtype: object

In [7]:
X = att.drop(["Attrition","BusinessTravel","Department","EducationField","Gender","MaritalStatus","OverTime"],axis=1)  # Assigning Explanatory Variable/Predictors
y = att[['Attrition']]              # Assigning Dependant variable/Target Variable
X.head()

,Age,EnvironmentSatisfaction,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,TotalWorkingYears,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,DistanceFromHome
0,41,2,3,2,4,5993,8,1,6,4,0,5,1
1,49,3,2,2,2,5130,10,3,10,7,1,7,8
2,37,4,2,1,3,2090,7,3,0,0,0,0,2
3,33,4,3,1,3,2909,8,3,8,7,3,0,3
4,27,1,3,1,2,3468,6,3,2,2,2,2,2


In [8]:
#one-hot encoding of Origin-categorical variable
temp = pd.get_dummies(att[["BusinessTravel","Department","EducationField","Gender","MaritalStatus","OverTime"]],prefix=["BusinessTravel","Department","EducationField","Gender","MaritalStatus","OverTime"]) 
temp.head() 

,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Human Resources,Department_Research & Development,Department_Sales,EducationField_Human Resources,EducationField_Life Sciences,EducationField_Marketing,EducationField_Medical,EducationField_Other,EducationField_Technical Degree,Gender_Female,Gender_Male,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,OverTime_No,OverTime_Yes
0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,1
1,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0,1,0
2,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,1
3,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,1
4,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,1,0,1,0


In [9]:
f = [X,temp] 
X = pd.concat(f,axis=1,join='inner')  #combing the one hot encoded categorical variable to rest of the predictors

### Train-Validation-Test Split

In [10]:
#For train-test split
from sklearn.model_selection import train_test_split   #import train_test_split function

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20,random_state=0)  # X-predictors, y-target

#Train-Validation Split
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.10,random_state=0)

### Desicion Tree Classifier

In [11]:
dt = DecisionTreeClassifier(random_state=0)

**Grid Search CV**

In [12]:
#Defining the parameters
param_grid = {'max_depth' : np.arange(1, 20, 1, dtype=int),'min_samples_leaf':np.arange(5, 100, 10, dtype=int)}

grid_search_cv = GridSearchCV(dt,param_grid,cv=10) # 10-fold cross validation
grid_search_cv.fit(X_train, y_train)               #fit grid search cv on training data

grid_search_cv.best_estimator_                     #best hyperparameters from gridsearch cv

DecisionTreeClassifier(max_depth=3, min_samples_leaf=45, random_state=0)

In [13]:
#Training the model with best hyperparameters
dt = DecisionTreeClassifier(max_depth=3, min_samples_leaf=45,random_state=0)
dt.fit(X_train,y_train)        #Fitting decision tree
pred1 = dt.predict(X_valid)

#accuracy score
print("The Accuracy of Decision tree with optimal hyperparameters on class-imbalanced data",accuracy_score(y_valid,pred1))

The Accuracy of Decision tree with optimal hyperparameters on class-imbalanced data 0.8135593220338984


In [14]:
#auroc score of decision tree with optimal hyperparameters
print("The AU-ROC score of Decision tree with optimal hyperparameters on class-imbalanced data",roc_auc_score(y_valid,pred1))

The AU-ROC score of Decision tree with optimal hyperparameters on class-imbalanced data 0.5875946969696969


### Random Forest Classifier 

In [15]:
rf = RandomForestClassifier()

**Grid Search CV**

In [ ]:
#Number of trees in random forest
n_estimators = np.arange(50, 950, 100, dtype=int)

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = np.arange(1, 15, 1, dtype=int)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]


# Create the GridSearchCv
param_grid = { 'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf':min_samples_leaf
             }

grid_search_cv = GridSearchCV(rf,param_grid,cv=10,n_jobs=4) # 10-fold cross validation
grid_search_cv.fit(X_train, y_train)               #fit grid search cv on training data

grid_search_cv.best_estimator_                     #best hyperparameters from gridsearch cv

In [ ]:
rf = RandomForestClassifier(max_depth=11, max_features='sqrt', n_estimators=200)
rf.fit(X_train,y_train)         #Fitting random forest model
pred2 = rf.predict(X_valid)    

#accuracy score
print("The Accuracy of Random Forest with optimal hyperparameters on class-imbalanced data",accuracy_score(y_valid,pred2))

In [ ]:
#auroc score of random forest with optimal hyperparameters
print("The AU-ROC score of Random Forest with optimal hyperparameters on class-imbalanced data",roc_auc_score(y_valid,pred2))

### Adaboost Classifier

In [ ]:
ada_boost = AdaBoostClassifier()

**Grid Search CV**

In [ ]:
# Create the GridSearchCv
param_grid = {'n_estimators':list(range(50, 102, 5)), 
              'learning_rate':[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]}

grid_search_cv = GridSearchCV(ada_boost,param_grid,cv=10) # 10-fold cross validation
grid_search_cv.fit(X_train, y_train)                      # fit grid search cv on training data

grid_search_cv.best_estimator_   

In [49]:
ada_boost = AdaBoostClassifier(learning_rate=0.8, n_estimators=100)
ada_boost.fit(X_train, y_train)
pred_ada = ada_boost.predict(X_valid)

#accuracy score
print("The Accuracy of Adaboost model with optimal hyperparameters on class-imbalanced data",accuracy_score(y_valid,pred_ada))

0.8389830508474576

In [50]:
#auroc score of adaboost with optimal hyperparameters
print("The AU-ROC score of Adaboost with optimal hyperparameters on class-imbalanced data",roc_auc_score(y_valid,pred_ada))
3.

0.6732954545454546

**Inference:** By fitting machine learning models like Decision Tree, Random Forest and Adaboost on Class-Imbalance dataset, it is found that Ada_Boost Model outperforms Random Forest and Decision Tree Model based on AU_ROC score.

_Thank You_